In [ ]:
# experimento com as imagens de todas as coletas da Palmer, utilizando 66% para treinamento e 33% para teste, para predição de SST

In [1]:
import cv2
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os 
import math

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split

%load_ext autoreload
plt.rcParams['figure.figsize'] = [20, 15]

path = os.getcwd()+'//..//imagens//'
sys.path.append(os.getcwd()+'//..//scripts//')
from MNG import MNG
from MNGFeatures import MNGFeatures
from MNGFeaturesMeans import MNGFeaturesMeans
from MNGModel import MNGModel

In [2]:
%autoreload 2

att = 'sst'
model_type = 'palmer'
palmer_sst = path + '//..//palmer_sst.csv'

In [3]:
# path = os.getcwd() + '//..//preprocessadas2//'
path = os.getcwd() + '//..//preprocessadas1//'
imgs = sorted(os.listdir(path))
img_names = [img.split('.')[0] for img in imgs]

mng = MNG(path, img_names, 'time', 20)

In [4]:
mng.features.dest_folder = os.getcwd() + '/..'
# mng.features.dest_folder = os.getcwd() + '//..//features//'

In [6]:
# nums = np.arange(1, 181)
nums = np.arange(1, 1321)

In [14]:
def compute_feature(current_features, features_name):
    mng.features.current_features = current_features
    mng.features.current_features_name = features_name
    mng.features.new_df()

    for img,num in zip(imgs, nums):
        print(num)
        bgr = cv2.imread(path+img)
        mng.features.extract_features(bgr, img)
    
    file_path = mng.features.save_data(False)

In [37]:
current_features = [mng.features.feature_names[:9], mng.features.feature_names[9:13], \
                    mng.features.feature_names[13], mng.features.feature_names[14:17],\
                    mng.features.feature_names[17], mng.features.feature_names[21:42],\
                    mng.features.feature_names[42:], mng.features.feature_names[18:21]]

features_names = ['means', 'size', 'dominant_HSV', 'rates', 'longitudinal_gradient',\
                  'rgb_3_regions', 'rgb_hsv_lab_n_regions', 'fractals']

In [9]:
current_features = mng.features.feature_names[18:21]

features_names = ['fractals']

In [15]:
for curr, feat in zip(current_features, features_names):
    compute_feature(curr, feat)

In [42]:
mng.model = MNGModel(path, file_path, model_type, att)
mng.model.build_mlr_model(model_type, features_mlr)

trees = [100, 200, 300, 400, 500, 600, 700]
for tree_num in trees:
    mng.model.build_rf_model(model_type, tree_num)

[0.01101535 0.01105941 0.89462016 0.01667592 0.01484859 0.00772028
 0.00612206 0.03145937 0.00647887]
[0.01528925 0.01133236 0.86263109 0.02499515 0.01571459 0.00982194
 0.0136343  0.03835928 0.00822203]
[0.01565045 0.01643335 0.82054462 0.03230305 0.02567242 0.00923339
 0.01624256 0.04844877 0.0154714 ]
[0.01341815 0.01364319 0.86405266 0.03122513 0.01700749 0.00904529
 0.01445426 0.02677317 0.01038067]
[0.01588813 0.01426283 0.8187836  0.02980261 0.02829098 0.0102489
 0.01157976 0.06055222 0.01059097]
[0.01366671 0.01975969 0.81605826 0.02393945 0.04499    0.00867667
 0.0151067  0.04395046 0.01385206]
[0.01024313 0.01259795 0.87850702 0.02085681 0.01354128 0.00899067
 0.01263427 0.03261116 0.01001772]


In [56]:
folder = '//features//'

In [58]:
data = pd.read_csv('//home//danielesreis//Projects//mng' + folder + 'size_all.csv', index_col=0, sep=';')

In [59]:
new_data = data.copy()
columns = data.columns
index = data.index.values

info = list()
info = [ind.split('_') for ind in index]

var = [inf[0][4:] for inf in info]
sem = [int(inf[1][3:]) for inf in info]
num = [int(inf[2].split('lado')[0][3:]) for inf in info]

var = pd.Series(var, index)
sem = pd.Series(sem, index)
num = pd.Series(num, index)

new_data['var'] = var
new_data['sem'] = sem
new_data['num'] = num

means = new_data.groupby(['var', 'sem', 'num'])[columns].mean()
new_data = means.reset_index().drop(columns=['var', 'sem', 'num'])

new_index = [ind.split('lado')[0] for ind in index]
new_index = new_index[::2]

new_data['ind'] = new_index
new_data.set_index('ind', inplace=True)

file_path = os.getcwd()+'//..' + folder + features_names[0] + '_all_half.csv'
new_data.to_csv(file_path, sep=';')

In [ ]:
# TESTE COM RGB APENAS

In [19]:
mng.features.add_target(file_path, palmer_time)

In [84]:
df_rgb = mng.model.data[['sst', 'mean_R_full', 'mean_G_full', 'mean_B_full']].copy()
df_rgb_path = path+'\\..\\features\\apenas_rgb.csv'
df_rgb.to_csv(df_rgb_path, sep=';')

In [75]:
mng.model = MNGModel(path, df_rgb_path, model_type, att)
mng.model.build_mlr_model(model_type, features_mlr)

trees = [100, 200, 300, 400, 500, 600, 700]
for tree_num in trees:
    mng.model.build_rf_model(model_type, tree_num)

[0.06636356 0.04893261 0.88470383]
[0.06429572 0.06301666 0.87268762]
[0.08230356 0.04243272 0.87526371]
[0.07087914 0.04359687 0.88552398]
[0.05384543 0.05248197 0.8936726 ]
[0.04399283 0.04615855 0.90984862]
[0.08021652 0.04966366 0.87011982]


In [ ]:
# TESTE COM HSV APENAS

In [81]:
df_hsv = mng.model.data[['sst', 'mean_H_full', 'mean_S_full', 'mean_V_full']].copy()
df_hsv_path = path+'\\..\\features\\apenas_hsv.csv'
df_hsv.to_csv(df_hsv_path, sep=';')

In [82]:
mng.model = MNGModel(path, df_hsv_path, model_type, att)
mng.model.build_mlr_model(model_type, features_mlr)

trees = [100, 200, 300, 400, 500, 600, 700]
for tree_num in trees:
    mng.model.build_rf_model(model_type, tree_num)

[0.06949251 0.7438529  0.18665459]
[0.07760629 0.7690112  0.15338251]
[0.05819901 0.78665167 0.15514932]
[0.05050865 0.80939203 0.14009932]
[0.05386391 0.78753751 0.15859858]
[0.05552808 0.79493086 0.14954107]
[0.05391616 0.77063927 0.17544457]


In [ ]:
# TESTE SEM SEMANA 4

In [53]:
sem_4 = mng.model.data.copy()
inds = mng.model.data.loc['0361palmer_sem4_num1':'0479palmer_sem4_num60'].index.values
sem_4.drop(inds, inplace=True)

sem_4_path = path+'\\..\\features\\sem_4.csv'
sem_4.to_csv(sem_4_path, sep=';')

In [54]:
mng.model = MNGModel(path, sem_4_path, model_type, att)
mng.model.build_mlr_model(model_type, features_mlr)

trees = [100, 200, 300, 400, 500, 600, 700]
for tree_num in trees:
    mng.model.build_rf_model(model_type, tree_num)